In [1]:
import pandas as pd
import json

In [2]:
df1 = pd.read_csv('review1.csv')
df1=df1.drop(['review_id','funny','cool'], axis=1)

In [3]:
df2 = pd.read_csv('review2.csv')
df2=df2.drop(['review_id','funny','cool'], axis=1)

In [4]:
df3 = pd.read_csv('review3.csv')
df3=df3.drop(['review_id','funny','cool'], axis=1)

In [5]:
df4 = pd.read_csv('review4.csv')
df4=df4.drop(['review_id','funny','cool'], axis=1)

In [6]:
df5 = pd.read_csv('review5.csv')
df5=df5.drop(['review_id','funny','cool'], axis=1)

In [7]:
df6 = pd.read_csv('review6.csv')
df6=df6.drop(['review_id','funny','cool'], axis=1)

In [8]:
df7 = pd.read_csv('review7.csv')
df7=df7.drop(['review_id','funny','cool'], axis=1)

In [9]:
df8 = pd.read_csv('review8.csv')
df8=df8.drop(['review_id','funny','cool'], axis=1)

In [10]:
df9 = pd.read_csv('review9.csv')
df9=df9.drop(['review_id','funny','cool'], axis=1)

In [11]:
df10 = pd.read_csv('review10.csv')
df10=df10.drop(['review_id','funny','cool'], axis=1)

In [12]:
df11 = pd.read_csv('review11.csv')
df11=df11.drop(['review_id','funny','cool'], axis=1)

In [13]:
df12 = pd.read_csv('review12.csv')
df12=df12.drop(['review_id','funny','cool'], axis=1)

In [14]:
df13 = pd.read_csv('review13.csv')
df13=df13.drop(['review_id','funny','cool'], axis=1)

In [15]:
df14 = pd.read_csv('review14.csv')
df14=df14.drop(['review_id','funny','cool'], axis=1)

In [16]:
df15 = pd.read_csv('review15.csv')
df15=df15.drop(['review_id','funny','cool'], axis=1)

In [17]:
df16 = pd.read_csv('review16.csv')
df16=df16.drop(['review_id','funny','cool'], axis=1)

In [18]:
df17 = pd.read_csv('review17.csv')
df17=df17.drop(['review_id','funny','cool'], axis=1)

In [19]:
df18 = pd.read_csv('review18.csv')
df18=df18.drop(['review_id','funny','cool'], axis=1)

In [20]:
#create a dataframe with all the reviews but without the text which is not relevant for the analysis
frames = [df1, df2, df3,df4, df5, df6,df7, df8, df9,df10, df11, df12,df13, df14, df15,df16, df17, df18]
df_r = pd.concat(frames)

In [21]:
df_r.head(2)

,user_id,business_id,stars,useful,text,date
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25


In [22]:
import numpy as np
import dask.bag as db
import dask.dataframe as dd # substitute of pandas to load the data in streams
import dask.array as da

In [23]:
df_r.head()

,user_id,business_id,stars,useful,text,date
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [24]:
#!pip install nltk

In [25]:
import nltk
import string
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Luca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
#n = len(df_r['text'])
df = df_r.sample(10000)
nt = []
for i in range(10000):
  s = str(df['text'].values[i])
  words = [word for word in s.split() if word.lower() not in sw_nltk]
  new_text = " ".join(words)
  new_text = new_text.translate(str.maketrans('', '', string.punctuation))
  nt.append(new_text.lower()) 
df['new_text'] = nt

In [28]:
df

,user_id,business_id,stars,useful,text,date,new_text
372339,g7NpwzTolxJUL4HLg0VbZQ,q3xo4nV19mqAizVJlRPmbw,1,1,Blue Host has been on a downward slide for ove...,2018-02-07 16:46:47,blue host downward slide year theyve obviously...
144885,kGEJs6k-5OcaugNtwhYLNQ,Q0sjLW9zqHLZ4p3xw166MQ,2,1,I was there a few years back. I appreciated th...,2017-06-16 14:44:38,years back appreciated video showed us explain...
107653,issfnrsMzmczjahtzNldaw,FbiExeeQVmpfCNo7YJuM9w,2,1,"Good place for nachos and drinks, that's about...",2012-08-22 11:17:29,good place nachos drinks thats it
379595,7LCfCuE5LkIxHRiJMNHdxg,DkKH_q0_vdWSWZwFhPFLZw,5,0,First - great service. My wife is pregnant an...,2018-02-04 20:15:43,first great service wife pregnant server poin...
245044,9549vDBw7ab0cuc930BQ1g,N_g-DECU6hOk7SJSCeCGzA,4,4,Friendly staff and is seemingly packed with us...,2007-06-24 09:31:41,friendly staff seemingly packed usuals food gr...
...,...,...,...,...,...,...,...
355071,OjmRp0Q5oGFLk7Uls6pUEw,AGwz-u4g80F8uEXJFPMxeA,5,0,Halinas staff is always professional and effic...,2017-06-28 13:20:37,halinas staff always professional efficient iv...
296070,c_SQlUe0mtAJSpK_LsMhIA,KOqTrsHbZXWnE7KdDS32gQ,4,0,This place may not impress people from the out...,2013-08-20 02:18:48,place may impress people outside cheesesteaks ...
379786,6WYegwqyZcI6C23eWnX6VQ,luDal00Gw3r-1KjI1454ZA,4,1,Great place for a night out with friends. Good...,2014-11-18 01:49:32,great place night friends good food good servi...
436895,r8loMAJix8rjhwKzwEVgDQ,o4kF_b-ChUzozUsfZ-eO4w,3,1,Delish but king crabs had Lil meat and not wor...,2018-08-29 23:47:29,delish king crabs lil meat worth 50 dollars 2 ...


In [29]:
#!pip install gensim

In [30]:
import gensim

In [31]:
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [33]:
data = df.new_text.values.tolist()
data_words = list(sent_to_words(data))

In [34]:
id2word = corpora.Dictionary(data_words)

In [35]:
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

In [36]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.012*"great" + 0.011*"food" + 0.008*"place" + 0.007*"good" + 0.006*"time" '
  '+ 0.006*"one" + 0.006*"would" + 0.006*"service" + 0.005*"get" + '
  '0.005*"like"'),
 (1,
  '0.008*"like" + 0.007*"one" + 0.006*"place" + 0.006*"time" + 0.006*"service" '
  '+ 0.005*"food" + 0.005*"great" + 0.005*"back" + 0.004*"good" + 0.004*"it"'),
 (2,
  '0.008*"great" + 0.008*"place" + 0.007*"get" + 0.007*"service" + '
  '0.006*"good" + 0.006*"would" + 0.006*"back" + 0.006*"food" + 0.006*"really" '
  '+ 0.005*"go"'),
 (3,
  '0.011*"food" + 0.011*"place" + 0.008*"great" + 0.007*"good" + 0.007*"one" + '
  '0.006*"go" + 0.005*"back" + 0.005*"like" + 0.005*"staff" + 0.004*"chicken"'),
 (4,
  '0.012*"place" + 0.008*"really" + 0.007*"great" + 0.007*"one" + 0.007*"like" '
  '+ 0.006*"time" + 0.006*"would" + 0.005*"service" + 0.005*"back" + '
  '0.005*"food"'),
 (5,
  '0.011*"good" + 0.010*"like" + 0.009*"food" + 0.007*"get" + 0.006*"really" + '
  '0.006*"time" + 0.006*"place" + 0.006*"service" + 0.006*

In [37]:
df=df.drop(['date','useful','user_id','business_id'],axis=1)   

In [38]:
for i in range(10):
    s=str(i)
    df[s]=np.zeros(len(df))
for i in range(len(df)):
    l=lda_model.get_document_topics(corpus[i])
    for a in l:
        df[str(a[0])]=a[1]

In [39]:
review_topics = lda_model.get_document_topics(corpus[100])

In [40]:
type(review_topics[0][0])

int

In [41]:
df.head(9)

,stars,text,new_text,0,1,2,3,4,5,6,7,8,9
372339,1,Blue Host has been on a downward slide for ove...,blue host downward slide year theyve obviously...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
144885,2,I was there a few years back. I appreciated th...,years back appreciated video showed us explain...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
107653,2,"Good place for nachos and drinks, that's about...",good place nachos drinks thats it,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
379595,5,First - great service. My wife is pregnant an...,first great service wife pregnant server poin...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
245044,4,Friendly staff and is seemingly packed with us...,friendly staff seemingly packed usuals food gr...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
458886,1,Picked this place for dinner over options in H...,picked place dinner options hilliard due ratin...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
164929,1,Canceled ny appointment half an hour before sc...,canceled ny appointment half hour scheduled s...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
114745,2,El Pollo Loco is a Mexican fast food chain tha...,el pollo loco mexican fast food chain competes...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
249906,5,I met Jimmy through a friend of mine and am so...,met jimmy friend mine happy ive mothers diamon...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891


In [42]:
#!pip install pyLDAvis

In [43]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./words_in_review2_'+ str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared,'./words_in_review2_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.007029 -0.000892       1        1  24.366157
4      0.000131 -0.004583       2        1  12.012201
7     -0.003752  0.003504       3        1  11.640087
2     -0.013981 -0.009705       4        1  10.432697
9     -0.006023  0.005254       5        1   9.033950
3     -0.011930  0.002772       6        1   8.614770
5      0.002704  0.007767       7        1   7.395799
6     -0.005460 -0.005771       8        1   7.259145
1      0.019438  0.024577       9        1   4.845062
8      0.025902 -0.022923      10        1   4.400132, topic_info=        Term         Freq        Total Category  logprob  loglift
61      good  4524.000000  4524.000000  Default  30.0000  30.0000
217     like  3325.000000  3325.000000  Default  29.0000  29.0000
104     food  4818.000000  4818.000000  Default  28.0000  28.0000
63     place  4576.000000  4576.000000  Default  27.0000  27.0000
203      get  3079.000000  3079.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
423   really    72.465910  2507.983009  Topic10  -5.8483  -0.4206
276      ive    64.736111  1710.454790  Topic10  -5.9611  -0.1507
290  ordered    61.809727  1499.801575  Topic10  -6.0074  -0.0655
393   little    60.106818  1433.427713  Topic10  -6.0353  -0.0482
464     went    57.525137  1441.899190  Topic10  -6.0792  -0.0980

[871 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
17769      6  0.758376  accolades
7049       1  0.234137  admission
7049       3  0.078046  admission
7049       4  0.078046  admission
7049       5  0.390228  admission
...      ...       ...        ...
1287       8  0.066887        yum
1287       9  0.041805        yum
1287      10  0.016722        yum
28328      3  0.721332     zafron
8126       2  0.662928       zola

[2406 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 8, 3, 10, 4, 6, 7, 2, 9])

In [55]:
df.to_pickle('df for topic analysis and predict rating')

In [5]:
df = pd.read_pickle('df for topic analysis and predict rating')

In [6]:
import sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

In [7]:
train, test = train_test_split(df, test_size=0.2)

In [8]:
train.head()

,stars,text,new_text,0,1,2,3,4,5,6,7,8,9
97776,2,I don't have especially high expectations for ...,especially high expectations bell every often ...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
401483,4,"We dined here because we love our kids, not be...",dined love kids expected good food buffet boat...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
272139,1,Didn't seem to even be able to get an appointm...,seem even able get appointment told time avail...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
335101,1,I have 3 credit cards with this bank never mis...,3 credit cards bank never missed payment whole...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
227820,5,"Strong margaritas.\nTender, flavorful beef faj...",strong margaritas tender flavorful beef fajita...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891


In [9]:
x_train, y_train = train.drop(columns = ['text', 'new_text', 'stars']), train['stars']

In [10]:
parameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [11]:
rf_model = RandomForestRegressor()

#we perform a grid search to find the best parameters in the param dictionary created before 
clf = RandomizedSearchCV(estimator= rf_model, param_distributions = parameters, n_iter = 20, scoring = 'neg_mean_squared_error', verbose=2, n_jobs=-1)
#we fit the model with the best parameters
clf.fit(x_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished


RandomizedSearchCV(estimator=RandomForestRegressor(), n_iter=20, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   scoring='neg_mean_squared_error', verbose=2)

In [12]:
x_test, y_test = test.drop(columns = ['text', 'new_text', 'stars']), test['stars']

In [13]:
y_pred = clf.predict(x_test)

In [14]:
MSE = np.mean((y_pred-y_test)**2)

In [15]:
MSE

2.1637048825043608

In [16]:
y_test

83248     5
450946    3
416841    5
240950    5
50892     3
         ..
269048    5
470891    1
154991    5
408391    5
341358    4
Name: stars, Length: 2000, dtype: int64

In [17]:
y_pred

array([3.73326458, 3.73326458, 3.73326458, ..., 3.73326458, 3.73326458,
       3.73326458])

In [20]:
from textblob import TextBlob

In [21]:
def getPolarity(text):
   return TextBlob(text).sentiment.polarity

In [22]:
df

,stars,text,new_text,0,1,2,3,4,5,6,7,8,9
372339,1,Blue Host has been on a downward slide for ove...,blue host downward slide year theyve obviously...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
144885,2,I was there a few years back. I appreciated th...,years back appreciated video showed us explain...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
107653,2,"Good place for nachos and drinks, that's about...",good place nachos drinks thats it,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
379595,5,First - great service. My wife is pregnant an...,first great service wife pregnant server poin...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
245044,4,Friendly staff and is seemingly packed with us...,friendly staff seemingly packed usuals food gr...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355071,5,Halinas staff is always professional and effic...,halinas staff always professional efficient iv...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
296070,4,This place may not impress people from the out...,place may impress people outside cheesesteaks ...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
379786,4,Great place for a night out with friends. Good...,great place night friends good food good servi...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891
436895,3,Delish but king crabs had Lil meat and not wor...,delish king crabs lil meat worth 50 dollars 2 ...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891


In [26]:
df['polarity'] = df['text'].apply(lambda x: getPolarity(x))

In [27]:
df

,stars,text,new_text,0,1,2,3,4,5,6,7,8,9,polarity
372339,1,Blue Host has been on a downward slide for ove...,blue host downward slide year theyve obviously...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,-0.200000
144885,2,I was there a few years back. I appreciated th...,years back appreciated video showed us explain...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.022511
107653,2,"Good place for nachos and drinks, that's about...",good place nachos drinks thats it,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.700000
379595,5,First - great service. My wife is pregnant an...,first great service wife pregnant server poin...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.506250
245044,4,Friendly staff and is seemingly packed with us...,friendly staff seemingly packed usuals food gr...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.587500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355071,5,Halinas staff is always professional and effic...,halinas staff always professional efficient iv...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.375000
296070,4,This place may not impress people from the out...,place may impress people outside cheesesteaks ...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.312500
379786,4,Great place for a night out with friends. Good...,great place night friends good food good servi...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.546875
436895,3,Delish but king crabs had Lil meat and not wor...,delish king crabs lil meat worth 50 dollars 2 ...,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,-0.075000


We train again our machine learning model with the additional feature

In [29]:
train, test = train_test_split(df, test_size=0.2)
x_train, y_train = train.drop(columns = ['text', 'new_text', 'stars']), train['stars']
x_test, y_test = test.drop(columns = ['text', 'new_text', 'stars']), test['stars']

In [30]:
x_train

,0,1,2,3,4,5,6,7,8,9,polarity
51588,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.477778
150680,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.120758
111775,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.265625
341257,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.210000
112575,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,-0.250000
...,...,...,...,...,...,...,...,...,...,...,...
88854,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.332143
244258,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,-0.362857
362121,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,0.215357
355749,0.53235,0.532248,0.930688,0.954971,0.143865,0.438007,0.394262,0.440094,0.381254,0.082891,-0.010606


In [31]:
rf_model = RandomForestRegressor()

#we perform a grid search to find the best parameters in the param dictionary created before 
clf = RandomizedSearchCV(estimator= rf_model, param_distributions = parameters, n_iter = 20, scoring = 'neg_mean_squared_error', verbose=2, n_jobs=-1)
#we fit the model with the best parameters
clf.fit(x_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.4min finished


RandomizedSearchCV(estimator=RandomForestRegressor(), n_iter=20, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   scoring='neg_mean_squared_error', verbose=2)

In [32]:
y_pred = clf.predict(x_test)
MSE = np.mean((y_pred-y_test)**2)
print(MSE)

1.1860093782853696


In [33]:
y_pred

array([4.35240547, 1.88548128, 4.43756448, ..., 1.17270517, 4.62865368,
       2.25668362])

In [36]:
round_pred = [round(el) for el in y_pred]

In [34]:
df_vis = pd.DataFrame({'true':y_test.values , 'pred': round})